In [102]:
import os
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras import metrics
from keras.optimizers import Adam
from time import time
from IPython.display import display, HTML

## Load Dataset

In [76]:
data_val = pd.read_csv('dataset/train_values.csv')
data_labels = pd.read_csv('dataset/train_labels.csv')

# Merge values and label
data = train_val.merge(data_labels, on='patient_id')
data = data.drop(columns='patient_id')

# Convert string values to integers
data["thal"] = data["thal"].astype('category')
data["thal"] = data["thal"].cat.codes

# Split dataset
train = data.sample(frac=0.8)
validate = data.sample(frac=0.2)

xsize = train.shape[1] - 1
ysize = 1
label_column = 'heart_disease_present'
cols = list(train.columns)
cols.remove(label_column)


In [104]:
display(train.head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present
132,1,1,130,2,0,0,2,204,1.4,0,41,172,0,0
105,2,1,130,4,2,0,0,303,2.0,0,64,122,0,0
54,1,1,130,4,0,0,2,330,0.0,0,61,169,0,1
42,1,1,110,3,1,1,2,265,0.0,0,71,130,0,0
111,1,1,120,2,0,0,0,244,1.1,0,50,162,0,0


In [12]:
# TODO: For Categorical Embeddings
cat_names = [
    'thal',
    'chest_pain_type',
    'num_major_vessels',
    'fasting_blood_sugar_gt_120_mg_per_dl',
    'resting_ekg_results',
    'sex',
    'exercise_induced_angina'
]
cont_names = [
    'slope_of_peak_exercise_st_segment',
    'resting_blood_pressure',
    'serum_cholesterol_mg_per_dl'
    'oldpeak_eq_st_depression',
    'age',
    'max_heart_rate_achieved',
    
]


## Define Model 

In [68]:
model = Sequential()
model.add(BatchNormalization(input_shape=(xsize,)))
model.add(Activation('relu'))
model.add(Dense(256))

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128))

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))

model.add(Dense(ysize))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=0.001),
    metrics=[metrics.binary_accuracy]
)

## Training

In [93]:
epochs = 100
batch_size = 32


print(train[cols].values[0])

model.fit(
    train[cols].values,
    train[label_column].values,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(validate[cols].values,validate[label_column].values)
)

# Save model and weights
save_dir=os.path.join(os.getcwd(), 'saved_models')
model_name='keras_mlwh_dnn_{}.h5'.format(time())
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

[  1.    1.  130.    2.    0.    0.    2.  204.    1.4   0.   41.  172.
   0. ]
Train on 144 samples, validate on 36 samples
Epoch 1/100
144/144 [==============================] - 0s 305us/step - loss: 0.0094 - binary_accuracy: 1.0000 - val_loss: 0.0081 - val_binary_accuracy: 1.0000
Epoch 2/100
144/144 [==============================] - 0s 380us/step - loss: 0.0302 - binary_accuracy: 0.9931 - val_loss: 0.0072 - val_binary_accuracy: 1.0000
Epoch 3/100
144/144 [==============================] - 0s 343us/step - loss: 0.0035 - binary_accuracy: 1.0000 - val_loss: 0.0063 - val_binary_accuracy: 1.0000
Epoch 4/100
144/144 [==============================] - 0s 333us/step - loss: 0.0123 - binary_accuracy: 1.0000 - val_loss: 0.0061 - val_binary_accuracy: 1.0000
Epoch 5/100
144/144 [==============================] - 0s 373us/step - loss: 0.0561 - binary_accuracy: 0.9861 - val_loss: 0.0065 - val_binary_accuracy: 1.0000
Epoch 6/100
144/144 [==============================] - 0s 343us/step - loss: 0.0

144/144 [==============================] - 0s 335us/step - loss: 0.0062 - binary_accuracy: 1.0000 - val_loss: 0.0046 - val_binary_accuracy: 1.0000
Epoch 52/100
144/144 [==============================] - 0s 330us/step - loss: 0.0081 - binary_accuracy: 1.0000 - val_loss: 0.0047 - val_binary_accuracy: 1.0000
Epoch 53/100
144/144 [==============================] - 0s 346us/step - loss: 0.0034 - binary_accuracy: 1.0000 - val_loss: 0.0053 - val_binary_accuracy: 1.0000
Epoch 54/100
144/144 [==============================] - 0s 292us/step - loss: 0.0091 - binary_accuracy: 1.0000 - val_loss: 0.0065 - val_binary_accuracy: 1.0000
Epoch 55/100
144/144 [==============================] - 0s 302us/step - loss: 0.0051 - binary_accuracy: 1.0000 - val_loss: 0.0084 - val_binary_accuracy: 1.0000
Epoch 56/100
144/144 [==============================] - 0s 345us/step - loss: 0.0078 - binary_accuracy: 1.0000 - val_loss: 0.0085 - val_binary_accuracy: 1.0000
Epoch 57/100
144/144 [==============================]

## Testing

In [105]:
test = pd.read_csv('dataset/test_values.csv')

# Convert string values to integers
test["thal"] = test["thal"].astype('category')
test["thal"] = test["thal"].cat.codes

predictions = model.predict(test[cols].values)

output = pd.DataFrame({
    'patient_id': test['patient_id'],
    'heart_disease_present': predictions[:,0]
})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(output)

,patient_id,heart_disease_present
0,olalu7,2.289770e-01
1,z9n6mx,3.470252e-04
2,5k4413,9.999391e-01
3,mrg7q5,1.880280e-06
4,uki4do,9.999870e-01
5,kev1sk,1.458649e-06
6,9n6let,8.683762e-06
7,jxmtyg,9.999837e-01
8,51s2ff,6.495454e-01
9,wi9mcs,1.209556e-01
